EPITA 2020 MLRF practice_02-03_twinit-part2-matching v2020-04-30_002250 by Joseph CHAZALON

<div style="overflow: auto; padding: 10px; margin: 10px 0px">
<img alt="Creative Commons License" src='img/CC-BY-4.0.png' style='float: left; margin-right: 20px'>
    
This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).
</div>

# Practice 02 part 03: Match keypoints and solve *Twin it!*
In this part we will reuse pre-computed elements from the previous parts:
- the distance matrix between bubbles computed from color histograms;
- keypoints and descriptors for each bubble.

The idea is to select pairs of bubbles which are close according to the color histogram, then to compare the descriptors extracted from each of them. Based on the number of near-identical matches, we will return a much compact list of twin candidates.

This last part is decomposed into 3 steps:
1. Prepare a matching framework to compare sets of descriptors.
2. Reload all the pre-computed elements from the previous parts.
3. Solve *Twin it!*.


## 1. Prepare a matching framework
Given two list of descriptors, $D_1$ and $D_2$ (which actually are flattened color image patches), we want to identify the matching pairs.

**Instead of using a distance like the sum of squared differences, we will use a scoring approach, therefore the higher the score the better the matching.**

This will be performed in three steps:
1. Find for each element $d_i \in D_1$ the best match $\hat{d_j} \in D_2$, ie build the set 
$$
\{
(d_i,\hat{d_j}) \mid
\hat{d_j} = \underset{d_j \in D_2}{\mathrm{argmax}} \operatorname{score}(d_i, d_j)
\},
$$
with the constraint that the matching score of two elements is above a minimal threshold: $$\operatorname{score}(d_i, d_j) > T.$$
In practice we only store the indices of the matching pairs.
2. Perform the reverse operation, find for each element $d_j \in D_2$ its best match $\hat{d_i} \in D_1$.
3. Keep only the matches which "agree", ie pairs that are in both sets.

**No need to use large descriptors to test this step:** we know our descriptors are 1-dimensional NumPy arrays, so you can test very simple cases to check your method before running it on large descriptors.

In [ ]:
# deactivate buggy jupyter completion
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [ ]:
# TODO
PATH_TO_RESOURCES = "."  # FIXME set this to the path of the twinit resource directory

#### Normalized cross correlation
The scoring method we will use to compare the descriptors (color image patches) had the following formula, where $d_i$ and $d_j$ are two descriptors of the same size (`3*patchside**2`):
$$
\operatorname{ncc}(d_i, d_j) = 
\frac{1}{|d_i|} 
\sum{
    \frac{d_i - \bar{d_i}}{\sigma_{d_i} + \epsilon}
    \times
    \frac{d_j - \bar{d_j}}{\sigma_{d_j} + \epsilon}
    }
$$
where:
- $|d_i| = |d_j|$ is the length of the descriptor;
- $\sum$ is the sum of the components of a vector;
- $\times$ is the component-wise product of two vectors;
- $\bar{d_i}$ is the mean value of $d_i$;
- $\sigma_{d_i}$ is the standard deviation of $d_i$;
- $\epsilon$ is a very small value ($\ll 1$) to avoid instability when $\sigma_{d_i} = 0$ (this may happens with buggy patches with constant values).

This simply compares vectors whose values are shifted around $0$ and scaled.

The result is close to $1$ for vectors which are highly colinears.

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Complete the function below to compute a normalized cross correlation between descriptors.**

Tip: Check Numpy documentation for your `np.array`s to find useful operations like `array.mean()` or `np.sum()`.
</div>

In [ ]:
# TODO complete this function
def ncc(v1, v2, epsilon=10e-6):
    '''Computes the normalized cross correlation between two vectors.'''
    n = len(v1)
    if n != len(v2):
        raise ValueError("v1 and v2 must have the same len."
                         "I got len(v1)=%d and len(v2)=%d" % (n, len(v2)))
    ncc_value = -1.  # FIXME
    return ncc_value

In [ ]:
# RUN ME
# Some tests to help you
print(ncc(np.arange(10), np.arange(10,20)))
print(ncc(np.arange(10), np.arange(20,10,-1)))

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Complete the functions below to compute matches between descriptors.**

Tips:
- Both functions returns the indices of the descriptors from `desc2` which are the best match to the descriptors from `desc1`, or `-1` if no suitable match is found.
- Test the matching without injecting your normalized cross correlation at first.
- `np.argsort(a)` gives the indices which sort `a`.
- `np.nonzero(bool_array)` gives the indices where `bool_array` is `True`.
</div>

In [ ]:
# TODO complete this function
def match(desc1, desc2, threshold=0.5):
    """ For each descriptor in the first set, 
        select its best match in the second set
        using normalized cross correlation.
        --
        Returns a list of the same size as desc1
        where elements are either an indice from descr2
        or -1 otherwise.
        """
    
    if len(desc1) == 0:
        return np.array([])
    if len(desc2) == 0:
        return np.full(len(desc1), -1)
    
    bestmatches = np.full(len(desc1), -1)  # FIXME
    
    return bestmatches

In [ ]:
# TODO complete this function
def match_twosided(desc1, desc2, threshold=0.5):
    """ Two-sided symmetric version of match().
        --
        Returns a list of the same size as desc1
        where elements are either an indice from descr2
        when symmetric match is verified,
        or -1 otherwise.
    """
    # Compute the matches
    # FIXME
    # matches_12 = ...
    # matches_21 = ...
    
    # remove matches that are not symmetric
    # FIXME
    
    return np.full(len(desc1), -1)  # FIXME

## 2. Reload everything and match some bubbles
**We are now ready to match descriptors for some bubbles!**

We will compare some bubbles using the descriptors we previously computed.

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Reload everything we need to match some bubbles, and solve the problem.**

We need:
- bubble images (color and grayscale),
- the distance matrix between bubbles computed using color histograms,
- the keypoints coordinates and descriptors we computed previously.
</div>

In [ ]:
# TODO load everything we need
bubble_files = !ls $PATH_TO_RESOURCES/bubbles_200dpi/b*.png | sort
bubbles = []  # FIXME
bubbles_gray = []  # FIXME
dist_mat = np.array([])  # FIXME
# TIP: use some values we computed for the distance matrix:
# npdata = np.load(PATH_TO_RESOURCES + "/bubble_dist_mat_rgb7-cosine.npz")
# dist_mat = npdata["dist_mat"]

keypoints = []  # FIXME
descriptors = []  # FIXME
# TIP: use some values we computed for keypoints and descriptors
# npdata = np.load(PATH_TO_RESOURCES + "/kpts_descr_harris_25pxcolor_mdist10.npz")
# keypoints = npdata["keypoints"]
# descriptors = npdata["descriptors"]

len(bubbles), len(bubbles_gray), dist_mat.shape, len(keypoints), len(descriptors)

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Using the display function provided below, compute and display some matches between a couple of bubbles.**

Tips:
- Bubbles with indices `35` and `219` are good candidates. So are bubbles `49` and `278`.
- Try to find a good value for the threshold.
</div>

In [ ]:
# Display functions
def appendimages(im1, im2):
    """ Return a new image that appends the two images side-by-side. """
    # select the image with the fewest rows and fill in enough empty rows
    rows1 = im1.shape[0]    
    rows2 = im2.shape[0]
    if rows1 < rows2:
        im1 = np.concatenate((im1, np.zeros((rows2-rows1,im1.shape[1]))),axis=0)
    elif rows1 > rows2:
        im2 = np.concatenate((im2, np.zeros((rows1-rows2,im2.shape[1]))),axis=0)
    # if none of these cases they are equal, no filling needed.
    return np.concatenate((im1,im2), axis=1)

def plot_matches(im_gray1, im_gray2, locs1, locs2, matches, show_below=True):
    """ Show a figure with lines joining the accepted matches 
        input: im_gray1,im_gray2 (images as arrays),
        locs1,locs2 (feature locations, aka keypoints), 
        matches (as output from 'match()'), 
        show_below (if images should be shown below matches). """
    if im_gray1.ndim != 2 or im_gray2.ndim != 2:
        raise ValueError("plot_matches takes gray images (ndim == 2) as arguments."
                         " I got im_gray1.ndim = %d and im_gray2.ndim = %d" 
                         % (im_gray1.ndim, im_gray2.ndim))
    im3 = appendimages(im_gray1, im_gray2)
    if show_below:
        im3 = np.vstack((im3,im3))
    plt.figure()
    plt.imshow(im3, cmap='gray')
    cols1 = im_gray1.shape[1]
    for i,m in enumerate(matches):
        if m >= 0:
            plt.plot([locs1[i][1],locs2[m][1]+cols1],[locs1[i][0],locs2[m][0]],'r')
    plt.axis('off')
    plt.show()

In [ ]:
# TODO display some matches, both single way and symmetric


<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Write down some observations about the previous matchings. What are the limitations of our approach?**
</div>

What are the limitations of the matching we implemented?

**TODO**

## 3. Solve *Twin it!*
At last we can try to filter bubbles more efficiently.

We will first pre-select the bubbles using the distance matrix computed using color histograms, then we will further filter this pre-selection using desriptor matching.
Then, we will be able to count the number of matches to select best candidates.

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Try to display only bubbles with twins. (Try to minimize the amount of human control.)**

Tips:
- For each bubble, display best candidates (if any).
- Keep only a few (5 or so) candidates using the distance matrix computed on color histograms.
- Use a restrictive threshold for descriptor matching (correlation > $0.9$).
- Use the count of matches to make a decision.
- Here are a few bubble ids to check if you do not have the time to run all the computation: `[0, 1, 35, 36, 43, 44, 49, 50, 91, 92, 105, 106]`.
</div>

In [ ]:
# TODO solve twin it!

# Job done!
You completed the session 2.

You should write down some observations below, like what are the parameters we tuned and how.

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Write some observations below.**

Tips:
- What are the parameters we tuned?
- Are there other parameters in our method?
</div>

TODO write some observations.